# Queries

### NOTE: All queries made within the month of January 2010, in the San Diego geographic area
1. What traffic station has the largest difference in average speed over the first two weeks of the month?
2. How significant is the difference in traffic throughput on a rainy Monday vs a non-rainy Monday?
3. Does trace amount of precipitation affect the number of CHP traffic incidents on a given day?  (Trace precipitation is defined as a weather station registering precipitation but less than the unit granularity of the sensor)
4. Identify the top 5 freeways with respect to traffic speed.
5. Is the traffic throughput of one freeway indicative of others?

In [3]:
import dbtemplate as dbt
import numpy as np
import pandas as pd
import pickle
from dbtemplate import StatementExecutorTemplateCallback
from dbtemplate import StatementExecutorTemplate

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
# login_info = {}
# login_info['db_name'] = 
# login_info['username'] = 
# login_info['password'] = 
# login_info['hostname'] = 
# login_info['port'] = 

# with open('pg_login.pkl', 'w') as pfile:
#     pickle.dump(login_info, pfile)

In [6]:
with open('pg_login.pkl') as pfile:
    login_info = pickle.load(pfile)
template = StatementExecutorTemplate(login_info['db_name'], login_info['username'], 
                                    login_info['password'], login_info['hostname'], login_info['port'])

# Results

## 1. What traffic station has the largest difference in average speed over the first two weeks of the month?

In [31]:
class InternalWeekOneCallback(StatementExecutorTemplateCallback):
    def _get_query(self):
        query= """
        SELECT t.pems_id, t.name, avg(o.avg_speed) AS week_one_avg_speed
        FROM traffic_station t
        INNER JOIN observation o on t.id = o.station_id
        WHERE o.time between '2010-01-01 00:00:00' AND '2010-01-07 23:59:59'
        GROUP BY t.pems_id, t.name
        HAVING avg(o.avg_speed) > 0
        """
        return query
    
    def _map_row(self, row):
        m_pems_id, m_traffic_station_name, week_one_avg_speed= row
        return (m_pems_id, m_traffic_station_name, float(week_one_avg_speed))

class InternalWeekTwoCallback(StatementExecutorTemplateCallback):
    def _get_query(self):
        query= """
        SELECT t.pems_id, t.name, avg(o.avg_speed) AS week_two_avg_speed
        FROM traffic_station t
        INNER JOIN observation o on t.id = o.station_id
        WHERE o.time between '2010-01-08 00:00:00' and '2010-01-14 23:59:59'
        GROUP BY t.pems_id, t.name
        HAVING avg(o.avg_speed) > 0
        """
        return query
    
    def _map_row(self, row):
        m_pems_id, m_traffic_station_name, week_two_avg_speed= row
        return (m_pems_id, m_traffic_station_name, float(week_two_avg_speed))

week_one_callback= InternalWeekOneCallback()
week_two_callback= InternalWeekTwoCallback()

m_week_one_list= template.execute(week_one_callback)
m_week_two_list= template.execute(week_two_callback)

week_one_df= dbt.to_data_frame(m_week_one_list, ['pems_id', 'traffic_station_name', 'week_one_avg_speed'])
week_two_df= dbt.to_data_frame(m_week_two_list, ['pems_id', 'traffic_station_name', 'week_two_avg_speed'])

week_one_df['week_one_avg_speed'] = week_one_df['week_one_avg_speed'].astype(float)
week_two_df['week_two_avg_speed'] = week_two_df['week_two_avg_speed'].astype(float)

if week_one_df is None or week_two_df is None:
    print "No Results"
else:
    m_working_df= pd.merge(week_one_df, week_two_df, on=['pems_id', 'traffic_station_name'])
    m_working_df['delta']= m_working_df['week_two_avg_speed'] - m_working_df['week_one_avg_speed']
    m_working_df.sort('delta', ascending=False, inplace=True)
    m_result= m_working_df.iloc[0]
    print m_result

pems_id                           1108491
traffic_station_name    WB MIRA MESA BLVD
week_one_avg_speed               63.32569
week_two_avg_speed               67.39831
delta                            4.072619
Name: 59, dtype: object


##### Traffic Station with the largest difference across the first two weeks of the month is WB MIRA MESA BLVD station

## 2. How significant is the difference in traffic throughput on a rainy Monday vs a non-rainy Monday?

In [7]:
m_all_mondays= ['2010-01-04', '2010-01-11', '2010-01-18', '2010-01-25']

In [15]:
class InternalRainyDayCallback(StatementExecutorTemplateCallback):
    
    def __init__(self, all_mondays):
        self.m_query_part= ""
        i= 0significant is the difference in traffic throughput on a rainy Monday vs a non-rainy Monday
        for d in all_mondays:
            if i > 0:
                self.m_query_part += ' or '
            self.m_query_part += "p.day = '" + str(d) + "'"
            i+=1
        StatementExecutorTemplateCallback.__init__(self)
        
        
    def _get_query(self):
        query= """
        select p.station_id, p.day, p.amount
        from precipitation_daily_total p
        where p.amount > 0 and ({})
        and p.station_id in (
            select id
            from weather_station
            where ST_Distance(ST_GeographyFromText('SRID=4267;POINT(-117.168347 32.711404)'), Location) < 15 * 1852
        )
        """
        s= query.format(self.m_query_part)
        return s
    
    def _map_row(self, row):
        station_id, day, amount= row
        return (station_id, day, float(amount))

rainy_day_callback= InternalRainyDayCallback(m_all_mondays)

rainy_day_list= template.execute(rainy_day_callback)
rainy_day_df= dbt.to_data_frame(rainy_day_list, ['station_id', 'day', 'amount'])

In [16]:
rainy_day_df

,station_id,day,amount
0,7740,2010-01-18,106


In [17]:
m_rainy_days= rainy_day_df['day'].values.tolist()
for d in m_rainy_days:
    print str(d)

2010-01-18


In [37]:
class InternalRainyDayObservationCallback(StatementExecutorTemplateCallback):
    def __init__(self, rainy_days):
        self.m_query_part= ""
        i= 0
        for d in rainy_days:
            if i > 0:
                self.m_query_part += ' or '
            self.m_query_part += "date(time) = '" + str(d) + "'"
            i+=1
        StatementExecutorTemplateCallback.__init__(self)
    
    def _get_query(self):
        query= """
        select avg(total_flow) as avg_rainy_day_throughput
        from observation
        where
        """
        query+= self.m_query_part
        return query
    
    def _map_row(self, row):
        avg_throughput= row[0]
        return float(avg_throughput)

rainy_day_obs_callback= InternalRainyDayObservationCallback(m_rainy_days)

rainy_day_obs_list= template.execute(rainy_day_obs_callback)
rainy_day_avg_throughput= rainy_day_obs_list[0]
print "rainy_day_avg_throughput=", rainy_day_avg_throughput, "v/5m"

throughput_per_day= rainy_day_avg_throughput * (12 * 24)
print "throughput_per_day=", throughput_per_day

rainy_day_avg_throughput= 113.602239918 v/5m
throughput_per_day= 32717.4450964


In [36]:
class InternalNonRainyDayObservationCallback(StatementExecutorTemplateCallback):
    def __init__(self, non_rainy_days):
        self.m_query_part= ""
        i= 0
        for d in non_rainy_days:
            if i > 0:
                self.m_query_part += ' or '
            self.m_query_part += "date(time) = '" + str(d) + "'"
            i+=1
        StatementExecutorTemplateCallback.__init__(self)
    
    def _get_query(self):
        query= """
        select avg(total_flow) as avg_non_rainy_day_throughput
        from observation
        where
        """
        query+= self.m_query_part
        return query
    
    def _map_row(self, row):
        avg_throughput= row[0]
        return float(avg_throughput)

non_rainy_days= set(m_all_mondays)
for d2 in m_rainy_days:
    d_str= str(d2)
    if d_str in non_rainy_days:
        non_rainy_days.remove(d_str)

non_rainy_day_obs_callback= InternalNonRainyDayObservationCallback(non_rainy_days)

non_rainy_day_obs_list= template.execute(non_rainy_day_obs_callback)
non_rainy_day_avg_throughput= non_rainy_day_obs_list[0]
print "non_rainy_day_avg_throughput=", non_rainy_day_avg_throughput, "v/5m"

throughput_per_day= non_rainy_day_avg_throughput * (12 * 24)
print "throughput_per_day=", throughput_per_day

non_rainy_day_avg_throughput= 141.788926919 v/5m
throughput_per_day= 40835.2109528


In [35]:
delta= non_rainy_day_avg_throughput - rainy_day_avg_throughput
print delta, "vehciles/5m"

delta_per_day= delta * (12 * 24)
print delta_per_day, "vehicles/day"

28.1866870013 vehciles/5m
8117.76585638 vehicles/day


##### The difference between a rainy Monday and a non-rainy Monday is significant, with an additional throughput of 8117.77 vehicles per day